### Import needed libraries(packages)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from sklearn.metrics import r2_score
from keras.initializers import HeUniform
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
%matplotlib inline

### Load csv file / read csv file

In [ ]:
Data_Frame = pd.read_csv('report.csv',delimiter=',')
Data_Frame.head(10)

### Select features

In [ ]:
Data_Frame_Features = Data_Frame[['CO','O3','NO2','SO2','PM10','PM2.5','AQI']]

### Train & Test split

In [ ]:
mask = np.random.rand(len(Data_Frame)) < 0.8
train = Data_Frame_Features[mask]
test = Data_Frame_Features[~mask]

### Standard scaling

In [ ]:
# Create your scaler model
Data_Scaler = StandardScaler()

# Convert x_train and x_test to numpy array
x_train = np.asanyarray(train[['CO','O3','NO2','SO2','PM10','PM2.5']]) 
x_test = np.asanyarray(Data_Frame_Features[['CO','O3','NO2','SO2','PM10','PM2.5']])

# Create your scaled x
scaled_x_train = Data_Scaler.fit(x_train).transform(x_train)
scaled_x_test = Data_Scaler.fit(x_test).transform(x_test)

### Create model(neural network)

In [ ]:
# Create you neural network model
model = Sequential()
model.add(Dense(1024,input_dim=scaled_x_train.shape[1],kernel_initializer=HeUniform(seed=0),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512,kernel_initializer=HeUniform(seed=0) ,activation='selu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(256,kernel_initializer=HeUniform(seed=0) ,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128,kernel_initializer=HeUniform(seed=0), activation='selu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1,kernel_initializer=HeUniform(seed=0) ,activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['mae'])

### Train model

In [ ]:
# Convert y_train and y_test to numpy array
y_train = np.asanyarray(train[['AQI']])
y_test = np.asanyarray(Data_Frame_Features[['AQI']])

# Train the model
Result = model.fit(scaled_x_train,y_train,epochs=100,batch_size=64,validation_split=0.2,verbose=1)
loss , mae = model.evaluate(scaled_x_test,y_test)
print(f'Test Mean Absolute Error : {mae:.2f}')

### Prediction

In [ ]:
# Predict with x_test
y_hat = model.predict(scaled_x_test)
print(y_hat)

### Prediction vs Reality(Plot y_hat vs y_test)

In [ ]:
figure , axs = plt.subplots(1,2,figsize=(12,4))
axs[0].scatter(y_test, y_hat)
axs[0].set_title('Real vs Predicted Values')
axs[0].set_xlabel('Real Values')
axs[0].set_ylabel('Predicted Values')

# Calculate the r2_score and plot it
r2 = r2_score(y_test, y_hat)
axs[1].text(0.5, 0.5, 'R-Squared: %.2f' % r2, fontsize=20, ha='center')
axs[1].set_title('R-Squared Value')
plt.show()

### List of predicted values and real values

In [ ]:
# Create an empty list to store the pairs
pairs = []

# Loop through the test set
for i in range(len(scaled_x_train)):
    # Create a tuple containing the real and predicted values
    pair = (y_test[i][0], y_hat[i][0])
    # (y_test,y_hat)
    
    # Append the tuple to the list
    pairs.append(pair)

# Print the list
for pair in pairs:
    print(pair)

### Plot MAE(mean absolute error)

In [ ]:
# Calculate the MAE for each pair of y_test and y_pred
mae_values = [mean_absolute_error(y_test[i], y_hat[i]) for i in range(len(y_test))]

# Define the range of x values for the plot
x_values = range(1, len(mae_values) + 1)

# Create a figure and an axis
fig, ax = plt.subplots()

# Plot the MAE values against the x values
ax.plot(x_values, mae_values)

# Set the x-axis label to 'Group' and the y-axis label to 'Mean Absolute Error'
ax.set_xlabel('Group')
ax.set_ylabel('Mean Absolute Error')

# Show the plot
plt.show()

### This neural network has been writen by me , Mohammad Parsa Rezaifar(MPR)
### This code is not great , so would be thankful if you develop it...
### Thanks to my dear friend , teamate Mr.Ahmad Moghadam(AM) to guide me :) 